<a href="https://colab.research.google.com/github/espickle1/boltz-2/blob/main/src/boltz_2_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#@title 1. Installing Boltz-2
#@markdown **If the program wants a restart,
#@markdown click on "RESTART RUNTIME" and let it restart. Move on to the next cell when restart is complete.
%%capture
!pip install boltz[cuda] -U

In [2]:
#@title 2. Import dependencies
!pip -q install ipywidgets
from google.colab import output, files
output.enable_custom_widget_manager()

import ipywidgets as W
from IPython.display import display, clear_output
import os, shlex, subprocess, sys, shutil, pathlib, time

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 44.4 MB/s eta 0:00:00


In [11]:
#@title 3. Upload prediction inputs
# 1) Upload local YAML(s)
print("Select and upload prediction YAML file:")
uploaded = files.upload()  # select one or more .yaml/.yml files

# 2) Small out_dir input + Run button
out_dir_txt = W.Text(
    value="/content/boltz_results",
    description="Output path:",
    layout=W.Layout(width="320px")  # compact width
)
run_btn = W.Button(description="Run boltz", button_style="success")
out = W.Output()


Select and upload prediction YAML file:


Saving t4_ev240_kdo3_template_affinity_v5.yaml to t4_ev240_kdo3_template_affinity_v5.yaml


In [12]:
#@title 4. Run Boltz-2 prediction
def stream_cmd(cmd_list):
    """
    Run a command and stream stdout+stderr line-by-line to the notebook output.
    Returns the process return code.
    """
    proc = subprocess.Popen(
        cmd_list,
        stdout=subprocess.PIPE,
        stderr=subprocess.STDOUT,
        text=True,
        bufsize=1,
        universal_newlines=True
    )
    for line in proc.stdout:
        print(line, end="")
        sys.stdout.flush()
    return proc.wait()

def zip_and_download(run_dir: pathlib.Path):
    """
    Zip the run_dir and trigger a download.
    """
    run_dir = run_dir.resolve()
    zip_base = str(run_dir)  # shutil.make_archive will add ".zip"
    zip_path = shutil.make_archive(zip_base, 'zip', root_dir=str(run_dir))
    print(f"\nZipped results → {zip_path}")
    files.download(zip_path)

def on_run(_):
    with out:
        clear_output()
        base_out_dir = pathlib.Path((out_dir_txt.value or "/content/boltz_results").strip())
        base_out_dir.mkdir(parents=True, exist_ok=True)

        for fname in uploaded.keys():
            yaml_path = pathlib.Path(fname).resolve()
            run_name = yaml_path.stem  # folder named after the YAML file
            run_dir = (base_out_dir / run_name)
            run_dir.mkdir(parents=True, exist_ok=True)

            print(f"\n=== Running: boltz predict on {yaml_path.name} ===")
            cmd = [
                "boltz", "predict",
                str(yaml_path),
                "--out_dir", str(run_dir),
                "--use_msa_server",
                "--recycling_steps", "12",
                "--sampling_steps", "512",
                "--diffusion_samples", "12",
                "--step_scale", "1.5",
                "--sampling_steps_affinity", "200",
                "--diffusion_samples_affinity", "12"
            ]
            print("$", " ".join(shlex.quote(x) for x in cmd))

            rc = stream_cmd(cmd)
            print(f"\n[boltz exited with code {rc}]")
            if rc != 0:
                print("⚠️ Command failed. Not zipping/downloading this run.")
                continue

            # Auto-zip and download this run's outputs
            try:
                # If tool writes asynchronously, tiny wait avoids missing files
                time.sleep(0.5)
                zip_and_download(run_dir)
            except Exception as e:
                print(f"⚠️ Could not zip/download results: {e}")

display(W.HBox([out_dir_txt, run_btn]))
display(out)
run_btn.on_click(on_run)


Output()